In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Sequential
import pickle

In [2]:
# Download required NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/anirudhb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anirudhb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anirudhb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#  Import and read the complaint_data.csv.
complaint_Data_df = pd.read_csv("../Resources/ModelData/train_test_data (1).csv")
complaint_Data_df.head()

,Product,Consumer complaint narrative
0,Credit card,16 date description debit 60 2 purchase 30 9 6...
1,Checking or savings account,due longer since notified dollar received leav...
2,Credit card,fcra asked due longer since please ive 19 plen...
3,Credit card,unwanted letter due additional alert received ...
4,Credit card,investigation also prehistoric issue possibly ...


In [4]:
#split the data

In [55]:
X = complaint_Data_df['Consumer complaint narrative']  # Features
y = complaint_Data_df['Product']  # Target labels

In [56]:
#X is our feature set and y is our target variable
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, #20% of the data is allocated to the test set, and 80% to the training set.
    stratify=y, #Ensures the distribution of y categories is preserved in the split.
    random_state=42
)

In [57]:
## Tokenize the text
# Define tokenizer with a vocabulary size of 25,000
tokenizer = Tokenizer(num_words=25000)

# Fit tokenizer on training data
tokenizer.fit_on_texts(X_train.values)

# Save the tokenizer
with open('../Resources/ModelData/tokenizer_3month_2product.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Transform training and testing text data into sequences of integers
train_sequences = tokenizer.texts_to_sequences(X_train.values)
test_sequences = tokenizer.texts_to_sequences(X_test.values)

# Print sample outputs for verification
print("Sample train sequence:", train_sequences[0])
print("Sample test sequence:", test_sequences[0])

Sample train sequence: [201, 75, 169, 703, 73, 667, 544, 63, 369, 3, 603, 355, 24, 687, 330, 559, 609, 688, 160, 204, 242, 247, 645, 121, 668, 200, 488, 93, 433]
Sample test sequence: [54, 27, 860, 67, 98, 16, 349, 4610, 102, 236, 24, 97, 365, 792, 616, 57, 280, 454, 700, 3784, 363, 623, 972, 301, 288, 999, 103, 449, 1111, 130, 1047, 410, 109, 95, 237, 185, 1095, 322, 1245, 129, 269, 110, 1, 14, 281, 26, 551, 3814, 872, 46, 108, 300, 239, 25, 971, 299, 49, 184, 401, 162, 52, 31, 80, 470, 196, 60, 910, 387, 59, 659, 666, 174, 1642, 78, 4, 219, 583, 6, 362, 1513, 121, 3040, 158, 29, 43, 529, 1287, 1182, 233, 71, 35, 920, 1104, 96, 354, 28, 18, 5707, 169, 2940, 1009, 471, 3, 38, 159, 3672, 11, 84, 1067, 76, 48, 1026, 128, 89, 7, 418, 51, 419, 20, 53, 5, 132, 545]


In [8]:
# Extract the word index (mapping of words to their respective indices)
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 14092 unique tokens.


In [9]:
vocabulary_size = len(tokenizer.word_index)+1 #where +1 accounts for an additional index for special tokens like padding or unknown tokens.
vocabulary_size

14093

In [10]:
# Get lengths of all sequences in the dataset
sequence_lengths = [len(text.split()) for text in X_train.values]
# Compute statistics
import numpy as np
print("Max length:", np.max(sequence_lengths))
print("Average length:", np.mean(sequence_lengths))
print("Median length:", np.median(sequence_lengths))
print("90th percentile:", np.percentile(sequence_lengths, 90))
print("95th percentile:", np.percentile(sequence_lengths, 95))

Max length: 718
Average length: 69.32749840865691
Median length: 56.0
90th percentile: 134.0
95th percentile: 168.0


In [11]:
# Pad sequences to ensure equal length(This approach reduces unnecessary padding while retaining most of the relevant data)
X_train_data = pad_sequences(train_sequences, maxlen=718, padding="post", truncating='post')#(Example maxlen based on our data analysis)
X_test_data = pad_sequences(test_sequences, maxlen=718, padding="post", truncating='post')#Truncates sequences longer than maxlen from the end.
print(f"X_train_data shape: {X_train_data.shape}")
print(f"X_test_data shape: {X_test_data.shape}")

X_train_data shape: (6284, 718)
X_test_data shape: (1572, 718)


In [12]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the training and test target variables
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Check the mapping of labels
print("Class Mapping:", dict(enumerate(label_encoder.classes_)))
print("Encoded y_train sample:", y_train_encoded[:5])

Class Mapping: {0: 'Checking or savings account', 1: 'Credit card'}
Encoded y_train sample: [1 1 1 1 1]


In [13]:
# Apply one-hot encoding to labels
labels_train = to_categorical(np.asarray(y_train_encoded))
labels_test = to_categorical(np.asarray(y_test_encoded))
# Print shapes again to verify
print('Shape of data tensor:', X_train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)

Shape of data tensor: (6284, 718)
Shape of label tensor: (6284, 2)
Shape of label tensor: (1572, 2)


In [14]:
# Build neural network with LSTM

In [16]:
model = Sequential()

# Add an embedding layer
model.add(Embedding(vocabulary_size,                # Vocabulary size
                    output_dim=300,                # Embedding size (vector size)
                    input_length=718,              # Maximum length of input sequences
                    trainable=True))               # Embeddings are trainable

# Add LSTM layer
#Bidirectional LSTM
model.add(Bidirectional(LSTM(300, dropout = 0.2, recurrent_dropout = 0.2)))

# Add Dense output layer for multiclass classification
model.add(Dense(2, activation='softmax'))  # 2 classes for multiclass

In [17]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(X_train_data, labels_train,
 batch_size=64,
 epochs=3,
 validation_data=(X_test_data, labels_test))

Epoch 1/3
99/99 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.6874 - loss: 0.5819 - val_accuracy: 0.8791 - val_loss: 0.2915
Epoch 2/3
99/99 ━━━━━━━━━━━━━━━━━━━━ 209s 2s/step - accuracy: 0.9270 - loss: 0.2072 - val_accuracy: 0.9122 - val_loss: 0.2482
Epoch 3/3
99/99 ━━━━━━━━━━━━━━━━━━━━ 209s 2s/step - accuracy: 0.9571 - loss: 0.1294 - val_accuracy: 0.8931 - val_loss: 0.2757


In [22]:
# Save the model
with open('../Resources/ModelData/LSTM_model_3month_2product.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
# Save the vectorizer
with open('../Resources/ModelData/tokenizer_3month_2product.pkl', 'wb') as vectorizer_file:
    pickle.dump(tokenizer, vectorizer_file)
print("LSTM and tokenizer saved with pickle.")

LSTM and tokenizer saved with pickle.


In [24]:
# Load the model
with open('../Resources/ModelData/LSTM_model_3month_2product.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)
# Load the vectorizer
with open('../Resources/ModelData/tokenizer_3month_2product.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

In [30]:
# Sample data
texts = '''I recently XXXX  a XXXX XXXX and I was in the XXXX  and also I am still seeing a XXXX XXXX  and taking tests. Because of my health problems I was unable to pay my bills on time. The bank added 10 late fees for insufficient funds. I asked for reimbursement but was denied. The fees are XXXX XXXX {$35.00} each= {$350.00}'''


# Tokenize the text
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences if necessary (depends on your model input requirements)
padded_sequences = pad_sequences(sequences, maxlen=718)


# Predict using the model
predictions = loaded_model.predict(padded_sequences)
print (predictions)

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 721ms/step
[[0.03463621 0.96536386]
 [0.02829746 0.9717025 ]
 [0.03383401 0.966166  ]
 [0.03345055 0.9665494 ]
 [0.03327135 0.9667287 ]
 [0.03345055 0.9665494 ]
 [0.035292   0.96470803]
 [0.02829746 0.9717025 ]
 [0.03457818 0.96542186]
 [0.034197   0.9658031 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.035292   0.96470803]
 [0.02829746 0.9717025 ]
 [0.02829746 0.9717025 ]
 [0.03463621 0.96536386]
 [0.02829746 0.9717025 ]
 [0.03460999 0.9653901 ]
 [0.02829746 0.9717025 ]
 [0.03458719 0.96541286]
 [0.02829

In [33]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(predictions, predictions.round())
print('precision: \n{}'.format(precision))
print('recall: \n{}'.format(recall))
print('fscore: \n{}'.format(fscore))
print('support: \n{}'.format(support))
Collapse

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multilabel-indicator targets

In [ ]:
print(classification_report(labels_test, predicted.round(),target_names=df1['product'].unique()))

In [59]:
# Assuming `model` is your trained model
model.save('LSTM_model_3month_2Products.h5')

from tensorflow.keras.models import load_model

# Load the tokenizer 
with open('../Resources/ModelData/tokenizer_3month_2product.pkl', 'rb') as handle: 
	tokenizer = pickle.load(handle)
	
# Load the model
model = load_model('LSTM_model_3month_2Products.h5')

# Single test example
text = '''problem asked fee 10 taking 00 denied bill fund health test 350 also 35 late reimbursement bank added unable insufficient time still seeing recently pay'''
words = text.split(" ")
print(len(words))

# Tokenize and pad the text
sequence = tokenizer.texts_to_sequences(text)
print(sequence)
X_test = pad_sequences(sequence, maxlen=718)  # Adjust maxlen based on your model's requirements
print(len(X_test[2]))
# Expected output (label)
y_test = np.array([1]*len(sequence)) 
#print(len(y_test))
rounded_labels=np.argmax(y_test, axis=1)
#rounded_labels[1]

# Example test data (replace with your actual test data)
import numpy as np

# Predict using the loaded model
predictions = model.predict(X_test)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming you are working on a classification problem
y_pred = (predictions > 0.5).astype("int32")  # Adjust threshold for your specific case
print(len(y_pred))
print(len(rounded_labels))
print(len(sequence))
# Calculate metrics
precision, recall, fscore, support = precision_recall_fscore_support(rounded_labels, y_pred, average='binary')
accuracy = accuracy_score(rounded_labels, y_pred)

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F-score: {fscore}')


25
[[2676], [2661], [], [744], [2465], [602], [7967], [], [], [1796], [3350], [602], [], [], [2111], [602], [602], [], [175], [551], [], [], [], [3350], [12895], [1637], [1719], [], [551], [551], [], [], [602], [1637], [12895], [602], [], [], [744], [12895], [2465], [2465], [], [2111], [73], [1637], [], [], [4895], [602], [], [2465], [], [4895], [], [], [602], [1796], [], [], [138], [258], [551], [], [], [2465], [1796], [], [], [138], [258], [], [2465], [], [], [602], [], [2661], [602], [12895], [7967], [744], [73], [2661], [1796], [602], [7967], [602], [1637], [], [], [744], [], [1637], [3350], [], [], [], [], [602], [], [], [73], [1637], [], [744], [2465], [602], [], [12895], [1637], [1796], [73], [2111], [2111], [12895], [330], [12895], [602], [1637], [], [], [], [12895], [7967], [602], [], [1796], [], [12895], [2465], [2465], [], [1796], [602], [602], [12895], [1637], [1719], [], [2661], [602], [330], [602], [1637], [], [2465], [11523], [], [2676], [], [11523]]
718


AxisError: axis 1 is out of bounds for array of dimension 1